### FastAPI Code

In [ ]:
import pandas as pd
import joblib
from fastapi import FastAPI
from pydantic import BaseModel

# Load the saved model (pipeline)
model = joblib.load('C:/Users/Administrator/Desktop/FastAPI/app/random_forest_pipeline.joblib')

# Initialize FastAPI app
app = FastAPI()

# Define the input data structure using Pydantic
class UserInput(BaseModel):
    Age: int
    Sex: str
    Cholesterol: int
    SystolicBP: int
    DiastolicBP: int
    HeartRate: int
    BMI: float
    Triglycerides: int
    Diabetes: int
    FamilyHistory: int
    Smoking: int
    Obesity: int
    AlcoholConsumption: int
    ExerciseHoursPerWeek: float
    Diet: str
    PreviousHeartProblems: int
    MedicationUse: int
    StressLevel: int
    SedentaryHoursPerDay: float

def preprocess_input(data):
    # Convert input to DataFrame for processing by the model pipeline.
    df = pd.DataFrame([data])
    
    # Perform feature engineering if necessary.
    df['Age_squared'] = df['Age'] ** 2
    df['Exercise_hours_squared'] = df['ExerciseHoursPerWeek'] ** 2
    
    # Handle missing columns by adding them with default values (0 or appropriate defaults)
    expected_columns = model.named_steps['preprocessor'].get_feature_names_out()
    
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0  # Default value for missing columns

    # Select only the columns the model expects.
    return df[expected_columns]

# Define the prediction endpoint
@app.post("/predict")
def predict_heart_attack_risk(input_data: UserInput):
    # Convert input data to a dictionary and preprocess it.
    input_dict = input_data.dict()
    
    # Preprocess and predict.
    processed_data = preprocess_input(input_dict)
    
    prediction = model.predict(processed_data)[0]
    
    # Return the prediction result.
    if prediction == 0:
        return {"Predicted Heart Attack Risk": "Low"}
    else:
        return {"Predicted Heart Attack Risk": "High"}

# Add a simple health check endpoint.
@app.get("/")
def read_root():
    return {"message": "Heart Attack Risk Prediction API is running"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
